In [2]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
learning_rate_RMSProp = 0.01
learning_rate_Gradient_Descent = 0.5
training_epochs = 100
softmax_classifier_iterations = 1000
batch_size = 256
display_step = 1
examples_to_show = 10
n_hidden_1 = 200
n_hidden_2 = 200
n_input = 784

In [0]:
# learning_rate_RMSProp = 0.01
# learning_rate_Gradient_Descent = 0.5
# training_epochs = 100
# softmax_classifier_iterations = 1000
# batch_size = 256
# desplay_step = 1
# examples_to_show = 10
# n_hidden_1 = 200
# n_hidden_2 = 200
# n_input = 784

In [0]:
# def build_autoencoder():
#   Wh_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
#   bh_1 = tf.Variable(tf.random_normal([n_hidden_1]))
#   h_1 = tf.nn.sigmoid(tf.matmul(X, Wh_1) + bh_1)
  
#   Wh_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
#   bh_2 = tf.Variable(tf.random_normal([n_hidden_2]))
#   h_2 = tf.nn.sigmoid(tf.matmul(h_1, Wh_2) + bh_2)
  
#   Wo = tf.Variable(tf.random_normal([n_hidden_2, n_input]))
#   bo = tf.Variable(tf.random_normal([n_input]))
#   X_reconstructed = tf.nn.sigmoid(tf.matmul(h_2, Wo) + bo)
  
#   return X_reconstructed, h_2

In [0]:
def build_autoencoder():
  Wh_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
  bh_1 = tf.Variable(tf.random_normal([n_hidden_1]))
  h_1 = tf.nn.sigmoid(tf.matmul(X, Wh_1) + bh_1)
  
  Wh_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
  bh_2 = tf.Variable(tf.random_normal([n_hidden_2]))
  h_2 = tf.nn.sigmoid(tf.matmul(h_1, Wh_2) + bh_2)
  
  Wo = tf.Variable(tf.random_normal([n_hidden_2, n_input]))
  bo = tf.Variable(tf.random_normal([n_input]))
  X_reconstructed = tf.nn.sigmoid(tf.matmul(h_2, Wo) + bo)
  
  return X_reconstructed, h_2

In [0]:
# def build_softmax_classifier():
#   W = tf.Variable(tf.zeros([n_hidden_2, 10]))
#   b = tf.Variable(tf.zeros([10]))
#   y_pred = tf.nn.softmax(tf.matmul(extracted_features, W) + b)
  
#   return y_pred

In [0]:
def build_softmax_classifier():
  W = tf.Variable(tf.zeros([n_hidden_2, 10]))
  b = tf.Variable(tf.zeros([10]))
  y_pred = tf.nn.softmax(tf.matmul(extracted_features, W) + b)
  
  return y_pred

In [0]:
# X = tf.placeholder('float', [None, n_input])
# y_pred, extracted_features = build_autoencoder()

# y_true = X
# y = build_softmax_classifier()
# y_ = tf.placeholder(tf.float32, [None, 10])

In [0]:
X = tf.placeholder('float', [None, n_input])
###y_pred = softmax result, extracted_features = hidden layer2 output
y_pred, extracted_features = build_autoencoder()

y_true = X
y = build_softmax_classifier()
y_ = tf.placeholder(tf.float32, [None, 10])

In [0]:
# reconstruction_cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
# initial_optimizer = tf.train.RMSPropOptimizer(learning_rate_RMSProp).minimize(reconstruction_cost)

In [0]:
# cross_entropy_cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# softmax_classifier_optimizer = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cross_entropy_cost)

In [0]:
# finetuning_cost = cross_entropy_cost + reconstruction_cost
# finetuning_optimizer = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(finetuning_cost)

In [0]:
reconstruction_cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
initial_optimizer = tf.train.RMSPropOptimizer(learning_rate_RMSProp).minimize(reconstruction_cost)

cross_entropy_cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
softmax_classifier_optimizer = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(cross_entropy_cost)

finetuning_cost = cross_entropy_cost + reconstruction_cost
finetuning_optimizer = tf.train.GradientDescentOptimizer(learning_rate_Gradient_Descent).minimize(finetuning_cost)

In [0]:
# print('Training Start...')
# import time

# start = time.time()
# total = time.time()

# with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
  
#   total_batch = int(mnist.train.num_examples / batch_size)
  
#   for epoch in range(training_epochs):
#     for i in range(total_batch):
#       batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#       _, cost_value = sess.run([initial_optimizer, reconstruction_cost], 
#                                feed_dict={X: batch_xs})
      
#     if epoch % 100 == 0:
#       print('Epoch step: {}, Loss: {}, Duration: {:.3f}'.format(epoch+1, cost_value, time.time() - start))
#       start = time.time()
#   print('pre-train optimization finished...')
  
#   reconstructed_image = sess.run(y_pred, feed_dict={X: mnist.test.images[:examples_to_show]})
#   f, a = plt.subplots(2, 10, figsize=(10, 2))
#   for i in range(examples_to_show):
#       a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
#       a[1][i].imshow(np.reshape(reconstructed_image[i], (28, 28)))
    
#   f.show()
#   plt.draw()
#   f.savefig('image.png')
  
#   for i in range(softmax_classifier_iterations):
#     batch_xs, batch_ys = mnist.train.next_batch(100)  
#     sess.run(softmax_classifier_optimizer, feed_dict={X: batch_xs, y_: batch_ys})
#   print('classifier optimization finisehd...')
  
#   correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
#   accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#   print('accuray(before fine-tuning): ')
#   print(sess.run(accuracy, feed_dict={X: mnist.test.images, y_: mnist.test.labels}))
  
#   start = time.time()
#   for epoch in range(training_epochs):
#     for i in range(total_batch):
#       batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#       _, cost_value = sess.run([finetuning_optimizer, finetuning_cost], feed_dict={X: batch_xs, y_: batch_ys})
      
# #       if epoch % 100 == 0:
# #         print('Epoch : {}, Loss: {}, Duration : {:.3f}'.format(epoch, cost_value, time.time() - start))
# #         start = time.time()
#   print('fine-tunintg finished...')
  
#   print('accuracy(after fine-tuning):')
#   print(sess.run(accuracy, feed_dict={X: mnist.test.images, y_: mnist.test.labels}))
#   print('Elapsed time: {:.3f}'.format(time.time() - total))
  

In [20]:
print('Training start...')
import time

start = time.time()
total = time.time()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  total_batch = int(mnist.train.num_examples / batch_size)
  
  for epoch in range(training_epochs):
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, cost_value = sess.run([initial_optimizer, reconstruction_cost], feed_dict = {X: batch_xs})
      
    if epoch % 50 == 0:
      print('Epoch step: {}, Loss: {:.5f}, Duration: {:.3f}'.format(epoch, cost_value, time.time() - start))
      
  print('pre-train optimization finished...')
  
#   reconstructed_image = sess.run(y_pred, feed_dict={X: mnist.test.images[:examples_to_show]})
#   f, a = plt.subplots(2, 10, figsize=(10, 2))
#   for i in range(examples_to_show):
#     a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
#     a[1][i].imshow(np.reshape(reconstructed_image[i], (28, 28)))
    
#   f.show()
#   plt.draw()
#   f.savefig('image.png')
  
  for i in range(softmax_classifier_iterations):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(softmax_classifier_optimizer, feed_dict={X: batch_xs, y_: batch_ys})
    
  print('classifier optimization finishied...')
  
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  accu = sess.run(accuracy, feed_dict = {X: mnist.test.images, y_: mnist.test.labels})
  print('accuracy(before fine-tuning): {}'.format(accu))
  
  start = time.time()
  for epoch in range(training_epochs):
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, cost_value = sess.run([finetuning_optimizer, finetuning_cost], feed_dict={X: batch_xs, y_: batch_ys})
  
    if epoch % 50 == 0:
      print('Epoch: {}, Loss: {}, Duration: {:.3f}'.format(epoch, cost_value, time.time() - start))
      start = time.time()
  
  print('fine-tuning finished...')
  
  accu = sess.run(accuracy, feed_dict={X: mnist.test.images, y_: mnist.test.labels})
  print('accuracy(after fine-tuning): {}'.format(accu))
  print('Elapsed time: {:.3f}'.format(time.time() - total))

Training start...
Epoch step: 0, Loss: 0.15401, Duration: 0.762
Epoch step: 50, Loss: 0.06633, Duration: 33.016
pre-train optimization finished...
classifier optimization finishied...
accuracy(before fine-tuning): 0.9132999777793884
Epoch: 0, Loss: 0.30908283591270447, Duration: 0.834
Epoch: 50, Loss: 0.13354361057281494, Duration: 38.529
fine-tuning finished...
accuracy(after fine-tuning): 0.9610999822616577
Elapsed time: 144.417
